In [1]:
import torch
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-160m')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'left'

tokenizer

GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-160m', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50256: AddedToken("                      ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50257: AddedToken("   

In [2]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset('imdb')
dataset = concatenate_datasets(list((dataset.values())))

f = lambda data: {
    'input_ids': tokenizer.encode(data['text'], truncation=True, max_length=5)
}
dataset = dataset.map(f, remove_columns=dataset.column_names)

dataset = dataset.train_test_split(test_size=2000)

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['input_ids'],
         num_rows: 98000
     })
     test: Dataset({
         features: ['input_ids'],
         num_rows: 2000
     })
 }),
 {'input_ids': [24221, 571, 16171, 1160, 617]})

In [3]:
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification

model_actor = AutoModelForCausalLM.from_pretrained('model/actor')
model_actor_ref = AutoModelForCausalLM.from_pretrained('model/actor')

model_critic = AutoModelForSequenceClassification.from_pretrained(
    'model/critic', num_labels=1)
model_critic_ref = AutoModelForSequenceClassification.from_pretrained(
    'model/critic', num_labels=1)

In [ ]:
from trl.trainer.ppov2_trainer import PPOv2Config, PPOv2Trainer
from transformers import TrainerCallback

config = PPOv2Config(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    total_episodes=20_0000,
    learning_rate=5e-6,
    logging_dir='output_dir',
    run_name='run_name',
    #non_eos_penalty=True,
    save_strategy='no')

trainer = PPOv2Trainer(config=config,
                       tokenizer=tokenizer,
                       policy=model_actor,
                       ref_policy=model_actor_ref,
                       reward_model=model_critic_ref,
                       value_model=model_critic,
                       train_dataset=dataset['train'],
                       eval_dataset=dataset['test'])
trainer.train()

model_actor.save_pretrained('model/trl')

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


===training policy===


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                ┃ score                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie is that it was made in the late eighties. The movie is │ 0.39717864990234375  │
│                          │ a perfect example of a movie that was made to make people     │                      │
│                          │ cry. The movie is a perfect example of how a movie can be     │                      │
│                          │ made to make people feel. The movie is a perfect example      │                      │
├──────────────────────────┼───────────────────────────────────────────────────────────────┼──────────────────────┤
│ This was a wonderful wit │ ty and entertaining movie. The acting was great, the plot was │ 1.0171935558319092   │
│                          │ great, and the music was great. I was surprised to see that   │                      │
│                          │ the movie was not as entertaining as the music. I was         │                      │
│                          │ surprised to see that the movie was not as good as the music. │                      │
├──────────────────────────┼───────────────────────────────────────────────────────────────┼──────────────────────┤
│ As I left the cinema     │  I turned to my friend and said, "Hey, I've got a movie to    │ 0.012245535850524902 │
│                          │ buy!", and I was like, "Great! Let's rent it!" I love horror  │                      │
│                          │ movies, especially those with a supernatural element. I was   │                      │
│                          │ like, "Great! Let                                             │                      │
├──────────────────────────┼───────────────────────────────────────────────────────────────┼──────────────────────┤
│ I picked this movie up   │  at my local Blockbuster for $1.50, and I'm glad I did. I was │ 0.958467423915863    │
│                          │ pleasantly surprised with the result. I was pleasantly        │                      │
│                          │ surprised by the acting, by the story, by the direction, and  │                      │
│                          │ by the cinematography. I                                      │                      │
├──────────────────────────┼───────────────────────────────────────────────────────────────┼──────────────────────┤
│ That is about the extent │  of the plot. The film is a series of disjointed, incoherent  │ -0.02634716033935547 │
│                          │ shots of a sprawling, sprawling mess of a movie. The film is  │                      │
│                          │ not a movie at all. It is a series of disjointed, incoherent  │                      │
│                          │ shots of a sprawling,                                         │                      │
└──────────────────────────┴───────────────────────────────────────────────────────────────┴──────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                  ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie is that it's a classic. This movie is a classic. This    │ 0.9531083106994629 │
│                          │ movie is a classic. This movie is a classic. This movie is a    │                    │
│                          │ classic. This movie is a classic. This movie is a classic. This │                    │
│                          │ movie is a classic. This movie is                               │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ This was a wonderful wit │ ty movie. It was a great movie. It was a great movie. It was a  │ 1.0156636238098145 │
│                          │ great movie. It was a great movie. It was a great movie. It was │                    │
│                          │ a great movie. It was a great movie. It was a great movie. It   │                    │
│                          │ was                                                             │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ As I left the cinema     │ , I thought that this was a great movie. It was a great movie.  │ 0.9743571281433105 │
│                          │ It was a great movie. It was a great movie. It was a great      │                    │
│                          │ movie. It was a great movie. It was a great movie. It was a     │                    │
│                          │ great movie. It                                                 │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ I picked this movie up   │  because it was a great movie. It was a great movie. It was a   │ 0.9922551512718201 │
│                          │ great movie. It was a great movie. It was a great movie. It was │                    │
│                          │ a great movie. It was a great movie. It was a great movie. It   │                    │
│                          │ was a great                                                     │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ That is about the extent │  of this movie. It's a great movie. It's a great movie. It's a  │ 0.9498776197433472 │
│                          │ great movie. It's a great movie. It's a great movie. It's a     │                    │
│                          │ great movie. It's a great movie. It's a great movie. It         │                    │
└──────────────────────────┴─────────────────────────────────────────────────────────────────┴────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                  ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie is the title. It's a classic. It's a classic. It's a     │ 0.9382727146148682 │
│                          │ classic. It's a classic. It's a classic. It's a classic. It's a │                    │
│                          │ classic. It's a classic. It's a classic. It's a                 │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ This was a wonderful wit │ . It was a good movie. It was a good movie. It was a good       │ 1.0170135498046875 │
│                          │ movie. It was a good movie. It was a good movie. It was a good  │                    │
│                          │ movie. It was a good movie. It was a good movie. It was a good  │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ As I left the cinema     │ , this was a good movie. It was a good movie. It was a good     │ 0.9625861048698425 │
│                          │ movie. It was a good movie. It was a good movie. It was a good  │                    │
│                          │ movie. It was a good movie. It was a good movie. It was a good  │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ I picked this movie up   │ . It was a good movie. It was a good movie. It was a good       │ 0.9509304165840149 │
│                          │ movie. It was a good movie. It was a good movie. It was a good  │                    │
│                          │ movie. It was a good movie. It was a good movie. It was a good  │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ That is about the extent │  of this movie. It's a classic. It's a classic. It's a classic. │ 0.9542768001556396 │
│                          │ It's a classic. It's a classic. It's a classic. It's a classic. │                    │
│                          │ It's a classic. It's a classic. It's a classic                  │                    │
└──────────────────────────┴─────────────────────────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                  ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie was the title. It was a good movie. It was a good movie. │ 0.9791495203971863 │
│                          │ It was a good movie. It was a good movie. It was a good movie.  │                    │
│                          │ It was a good movie. It was a good movie. It was a good movie.  │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ This was a wonderful wit │ . It was a great movie. It was a great message. It was a great  │ 1.0305516719818115 │
│                          │ message. It was a great message. It was a great message. It was │                    │
│                          │ a great message. It was a great message. It was a great         │                    │
│                          │ message. It was a great                                         │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ As I left the cinema     │ , I saw this movie. It was a pleasant surprise. It was a        │ 1.0189690589904785 │
│                          │ pleasant surprise. It was a pleasant surprise. It was a         │                    │
│                          │ pleasant surprise. It was a pleasant surprise. It was a         │                    │
│                          │ pleasant surprise. It was a pleasant surprise. It was a         │                    │
│                          │ pleasant surprise                                               │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ I picked this movie up   │  on DVD. It was a pleasant surprise. It was a pleasant          │ 1.0111174583435059 │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was                                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ That is about the extent │ . It was a great movie. It was a great movie. It was a great    │ 0.9807864427566528 │
│                          │ movie. It was a great movie. It was a great movie. It was a     │                    │
│                          │ great movie. It was a great movie. It was a great movie. It was │                    │
│                          │ a great                                                         │                    │
└──────────────────────────┴─────────────────────────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                  ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie is the title. It was a great movie. It was a great       │ 1.0070037841796875 │
│                          │ message. It was a great message. It was a great message. It was │                    │
│                          │ a great message. It was a great message. It was a great         │                    │
│                          │ message. It was a great message.                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ This was a wonderful wit │ ful movie. It was a pleasant surprise. It was a pleasant        │ 1.025543451309204  │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was                                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ As I left the cinema     │ , this was a pleasant surprise. It was a pleasant surprise. It  │ 0.9695231914520264 │
│                          │ was a pleasant surprise. It was a pleasant surprise. It was a   │                    │
│                          │ pleasant surprise. It was a pleasant surprise. It was a         │                    │
│                          │ pleasant surprise. It was a pleasant surprise. It was a         │                    │
│                          │ pleasant                                                        │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ I picked this movie up   │  on DVD. It was a pleasant surprise. It was a pleasant          │ 1.0111174583435059 │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was                                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ That is about the extent │  of this movie. It was a pleasant surprise. It was a pleasant   │ 0.9984803199768066 │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It                                                    │                    │
└──────────────────────────┴─────────────────────────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                  ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie is the title. It's a great movie. It's a great movie.    │ 0.9969281554222107 │
│                          │ It's a great movie. It's a great movie. It's a great movie.     │                    │
│                          │ It's a great movie. It's a great movie. It's a great movie.     │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ This was a wonderful wit │ ful movie. It was a great movie. It was a great movie. It was a │ 1.0189448595046997 │
│                          │ great movie. It was a great movie. It was a great movie. It was │                    │
│                          │ a great movie. It was a great movie. It was a great movie. It   │                    │
│                          │ was                                                             │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ As I left the cinema     │ , I saw this movie in the theater. It was a pleasant surprise.  │ 1.0127575397491455 │
│                          │ It was a pleasant surprise. It was a pleasant surprise. It was  │                    │
│                          │ a pleasant surprise. It was a pleasant surprise. It was a       │                    │
│                          │ pleasant surprise. It was a pleasant surprise. It was           │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ I picked this movie up   │  on DVD. It was a pleasant surprise. It was a pleasant          │ 1.0111174583435059 │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was                                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ That is about the extent │  of this movie. It was a great movie. It was a great message.   │ 0.9768314361572266 │
│                          │ It was a great message. It was a great message. It was a great  │                    │
│                          │ message. It was a great message. It was a great message. It was │                    │
│                          │ a great message. It                                             │                    │
└──────────────────────────┴─────────────────────────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                    ┃ model response                                                  ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ The best part about this │  movie is the title. It was a great movie. It was a great       │ 1.0102629661560059 │
│                          │ story. It was a great message. It was a great message. It was a │                    │
│                          │ great message. It was a great movie. It was a great movie. It   │                    │
│                          │ was a great movie.                                              │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ This was a wonderful wit │ ful movie. It was a pleasant surprise. It was a pleasant        │ 1.025543451309204  │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was                                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ As I left the cinema     │ , I felt that this was a great movie. It was a great message.   │ 0.9691681265830994 │
│                          │ It was a great message. It was a great message. It was a great  │                    │
│                          │ message. It was a great message. It was a great message. It was │                    │
│                          │ a great message. It                                             │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ I picked this movie up   │  on DVD. It was a pleasant surprise. It was a pleasant          │ 1.0111174583435059 │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was                                                │                    │
├──────────────────────────┼─────────────────────────────────────────────────────────────────┼────────────────────┤
│ That is about the extent │  of this movie. It was a pleasant surprise. It was a pleasant   │ 0.9984803199768066 │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It was a pleasant surprise. It was a pleasant         │                    │
│                          │ surprise. It                                                    │                    │
└──────────────────────────┴─────────────────────────────────────────────────────────────────┴────────────────────┘